In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# importing libraries
from pathlib import Path
import glob
import pandas as pd
import cv2

# Prepare Training Set

In [2]:
#0 for compostable
#1 for recyclable 

# reading images from path
images_dir = Path('O')
images = images_dir.glob("*.jpg")


train_data1 = []

for img in images:
    imgUMat = cv2.imread(str(img))
    train_data1.append((imgUMat,0))

 
# converting data into pandas dataframe for easy visualization 
train_data1 = pd.DataFrame(train_data1,columns=['image','label'],index = None)

In [3]:
# reading images from path
images_dir = Path('R')
images2 = images_dir.glob("*.jpg")

train_data2 = []

for img in images2:
    imgUMat = cv2.imread(str(img))
    train_data2.append((imgUMat,1))
 
# converting data into pandas dataframe for easy visualization 
train_data2 = pd.DataFrame(train_data2,columns=['image','label'],index = None)

In [4]:
train_data = pd.concat([train_data1, train_data2])

In [5]:
train_data = train_data.reset_index(drop=True)

In [20]:
# Preprocessing
def processing(data):
    # setting dim of the resize
    height = 100
    width = 100
    dim = (width, height)
    res_img = []
    for i in range(len(data)):
        res = cv2.resize(data.loc[i, 'image'], dim, interpolation=cv2.INTER_AREA)
        res_img.append(res)
    no_noise = []
    for i in range(len(res_img)):
        blur = cv2.GaussianBlur(res_img[i], (5, 5), 0)
        blur = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
        no_noise.append(blur)
    return no_noise

In [21]:
new_img = processing(train_data)

In [23]:
X = np.reshape(new_img,(22564, 10000))

In [24]:
X = np.concatenate([X[:1000], X[21564:]])

In [25]:
X.shape

(2000, 10000)

# Fit Training Set

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [27]:
y= np.array(train_data['label'])

In [28]:
y = np.concatenate([y[:1000], y[21564:]])

In [29]:
y.shape

(2000,)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5, random_state=42)

In [31]:
kn_clf = KNeighborsClassifier()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

In [32]:
rnd_clf.fit(X_train, y_train)

RandomForestClassifier()

In [33]:
voting_clf = VotingClassifier(estimators=[('kn', kn_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting='soft')

In [34]:
from sklearn.metrics import accuracy_score

for clf in (kn_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

KNeighborsClassifier 0.697
RandomForestClassifier 0.783
SVC 0.768
VotingClassifier 0.78


In [42]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200, algorithm='SAMME.R', learning_rate=0.5)

ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

In [43]:
y_pred = ada_clf.predict(X_test)

In [44]:
accuracy_score(y_test, y_pred)

0.748